In [1]:
# Comparing using some data vs all data when not much has been collected yet.
# Goal is to choose between many sampled models or one model with many agents.

In [3]:
import multiprocessing 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import copy

import model_based_agent as mba 
import utils as ut
import tab_agents as tab

from datetime import datetime 

In [13]:
def plot_qtabs(agt,vmin=None,vmax=None):
    fig,ax = plt.subplots(1,2)
    for i in range(2):
        img = ax[i].imshow(agt.Qtab[:,i].reshape(12,12),vmin=vmin,vmax=vmax)
        fig.colorbar(img,ax=ax[i])

In [11]:
num_learners = 10
collection_eps = 10

gamma=.25
epsilon=.05
alpha=.01
init_df = 'ensemble_testing.pkl'
qtabs = []

if __name__=='__main__':
    
    '''
    Runs parallel learners and joins them afterward
    '''

    worm_agent = tab.Q_Alpha_Agent(gamma=gamma, epsilon=epsilon, alpha=alpha) # Agent doesn't learn
    # Start model environments and learners 
    dh = mba.DataHandler()
    dh.load_df(init_df)
    dh.df = dh.df[:2000]
    
    for loop in range(collection_eps):        
        # Make new learner list based on newest data
        learners = mba.make_learner_list(num_learners, worm_pars={'num_models':1, 'frac':1},
                                gamma=gamma, epsilon=epsilon, alpha=alpha, q_checkpoint=worm_agent.Qtab)        
        
        # Start multiprocessing
        manager = multiprocessing.Manager()
        poison_queue = manager.Queue()
        pool = multiprocessing.Pool()
        
        # Run main functions: train agents and collect more data
        lea_outs = []
        print('learning')
        for lea in learners:
            lea_outs.append(pool.apply_async(lea.learn, [dh],{'learn_limit':100}))
        # Wait for them to finish
        pool.close()
        pool.join()
        for lea_out in lea_outs:
            if lea_out.successful() is False:
                raise Exception('One learner failed')
        
        # Combining learners from run
        worm_agent.Qtab = mba.combine_learners(lea_outs)
        qtabs.append(worm_agent.Qtab.reshape((12,12,2)))

learning


Exception: One learner failed